# The Keras Functional API in TensorFlow
函数式的API，给予了Keras更多的flexible，而不是只是通过sequence，去堆叠神经网络了。

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

tf.keras.backend.clear_session()  # For easy reset of notebook state.

上节课我们已经了解了如何通过keras.Model()去自定义层了。
1. 先建立层结构（kernel结构和数据流结构）
2. 放在model里面


In [2]:
# 建立结构
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu',name='GOOD')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

#放在model里面
model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [3]:
print(model.inputs)

[<tf.Tensor 'img:0' shape=(None, 784) dtype=float32>]


In [4]:
#看看什么结构
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 784)]             0         
_________________________________________________________________
GOOD (Dense)                 (None, 64)                50240     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


none表示input的维度是不固定的，会随着上一层的input变换而变换。

## training evaluation and inference

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

W0922 22:28:15.973103 4634564032 deprecation.py:323] From /Users/allen/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 38us/sample - loss: 0.3605 - accuracy: 0.8986 - val_loss: 0.1983 - val_accuracy: 0.9424
Epoch 2/5
48000/48000 [==============================] - 2s 34us/sample - loss: 0.1660 - accuracy: 0.9506 - val_loss: 0.1446 - val_accuracy: 0.9582
Epoch 3/5
48000/48000 [==============================] - 2s 47us/sample - loss: 0.1208 - accuracy: 0.9640 - val_loss: 0.1214 - val_accuracy: 0.9645
Epoch 4/5
48000/48000 [==============================] - 2s 36us/sample - loss: 0.0962 - accuracy: 0.9712 - val_loss: 0.1170 - val_accuracy: 0.9657
Epoch 5/5
48000/48000 [==============================] - 2s 35us/sample - loss: 0.0793 - accuracy: 0.9768 - val_loss: 0.1061 - val_accuracy: 0.9698
Test loss: 0.09988791370857507
Test accuracy: 0.9695


## 用同样的layer来定义多种model
如果我们用kera.Model来定义神经网络结构，那数据的流动才是决定神经网络结构的关键。至于怎么写，并不重要

In [6]:
#首先我们定义了神经网络结构为，
#从encoder_input 到 encoder_output

encoder_input = keras.Input(shape=(28, 28, 1), name='img')
x = layers.Conv2D(16, 3, activation='relu')(encoder_input)
x = layers.Conv2D(32, 3, activation='relu')(x)
mid_break = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation='relu')(mid_break)
x = layers.Conv2D(16, 3, activation='relu')(x)
encoder_output = layers.GlobalMaxPooling2D()(x)


#看看从encoder_input 到 encoder_output的结构
encoder = keras.Model(encoder_input, encoder_output, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                0   

__我们尝试以mid_break 作为输出，那么神经网络的结构也就被拆分开了。__

In [7]:
mid_break= keras.Model(encoder_input, mid_break, name='encoder')
mid_break.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
Total params: 4,800
Trainable params: 4,800
Non-trainable params: 0
_________________________________________________________________


__因此，根据上面的分析，只要我们把数据流给他连接上，就可以接着上面的结构继续设计了。__

In [8]:
# 将encoder_output接着往下写
x = layers.Reshape((4, 4, 1))(encoder_output)

x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation='relu')(x)

#我们规定的input端不变（input端必须有spaceholder），输出端变成了decoder_output，那么网络结构也就跟着写下来了

autoencoder = keras.Model(encoder_input, decoder_output, name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                

### 把model当成一个层来使用
keras中，我们可以把一个model当成一个层，给input和output，就能把model当层来使用。其他的和keras.Model的用法相同。

__先建立一个model1__

In [9]:
input1 = keras.Input(shape=(28, 28, 1), name='original_img')
x = layers.Conv2D(16, 3, activation='relu')(input1)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
output1 = layers.GlobalMaxPooling2D()(x)

model1 = keras.Model(input1, output1, name='model1')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                0   

__在建立一个model2__

In [10]:
input2 = keras.Input(shape=(16,), name='model2')
x = layers.Reshape((4, 4, 1))(input2)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
output2 = layers.Conv2DTranspose(1, 3, activation='relu')(x)

model2= keras.Model(input2, output2, name='model2')
model2.summary()

Model: "model2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model2 (InputLayer)          [(None, 16)]              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 1)           0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 6, 6, 16)          160       
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 8, 8, 32)          4640      
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 26, 26, 16)        4624      
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 28, 28, 1)         145  

__我要把model1和model2 就像一个layer一样使用，建立一个新的model3__

In [11]:
input3 = keras.Input(shape = (28,28,1), name ='imag')

# 利用model1 和 model2 的数据的首位想接，完成两个model的合并。
model1_output = model1(input3)
model2_output = model2(model1_output)

output3 = model2_output

model3 = keras.Model(input3, output3, name='model3')

model3.summary()


Model: "model3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
imag (InputLayer)            [(None, 28, 28, 1)]       0         
_________________________________________________________________
model1 (Model)               (None, 16)                18672     
_________________________________________________________________
model2 (Model)               (None, 28, 28, 1)         9569      
Total params: 28,241
Trainable params: 28,241
Non-trainable params: 0
_________________________________________________________________


__model1 和 model2 就成立model3 的submodel了，其作用和一个层是一样的。__

## 模型的并联
模型的并联是一种最简单的控制数据流的方法了。就是将同样的input输入到三个模型中（三个模型结构可以相同也可以不同），而后将三个模型的输出，拿出来，放到平均层中去取平均。

In [12]:
def get_model():
    inputs = keras.Input(shape=(128,))
    outputs = layers.Dense(1, activation='sigmoid')(inputs)
    return keras.Model(inputs, outputs)

#随便定义三个结构相同的模型
model1 = get_model()
model2 = get_model()
model3 = get_model()


# 给出一个input的spaceholder
inputs = keras.Input(shape=(128,))

# 将这个共同的inputs分别赋值给三个model。得到三个（一维）输出
y1 = model1(inputs)
y2 = model2(inputs)
y3 = model3(inputs)


#将这3个输出，作为输入放在average层里取平均
outputs = layers.average([y1, y2, y3])

#数据的流从inputs，经过三条路，分别进入三个模型中，三个模型的输出最后合并到一起了，
# 进入到average 层 而后输出为outputs
ensemble_model = keras.Model(inputs=inputs, outputs=outputs)

In [13]:
#看一下他的结构
ensemble_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
model (Model)                   (None, 1)            129         input_4[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            129         input_4[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 1)            129         input_4[0][0]                    
____________________________________________________________________________________________

In [14]:
num_tags = 12  # Number of unique issue tags
num_words = 10000  # Size of vocabulary obtained when preprocessing text data
num_departments = 4  # Number of departments for predictions

title_input = keras.Input(shape=(None,), name='title')  # Variable-length sequence of ints
body_input = keras.Input(shape=(None,), name='body')  # Variable-length sequence of ints
tags_input = keras.Input(shape=(num_tags,), name='tags')  # Binary vectors of size `num_tags`

# Embed each word in the title into a 64-dimensional vector
title_features = layers.Embedding(num_words, 64)(title_input)
# Embed each word in the text into a 64-dimensional vector
body_features = layers.Embedding(num_words, 64)(body_input)

# Reduce sequence of embedded words in the title into a single 128-dimensional vector
title_features = layers.LSTM(128)(title_features)
# Reduce sequence of embedded words in the body into a single 32-dimensional vector
body_features = layers.LSTM(32)(body_features)

# Merge all available features into a single large vector via concatenation
x = layers.concatenate([title_features, body_features, tags_input])

# Stick a logistic regression for priority prediction on top of the features
priority_pred = layers.Dense(1, activation='sigmoid', name='priority')(x)
# Stick a department classifier on top of the features
department_pred = layers.Dense(num_departments, activation='softmax', name='department')(x)

# Instantiate an end-to-end model predicting both priority and department
model = keras.Model(inputs=[title_input, body_input, tags_input],
                    outputs=[priority_pred, department_pred])

In [16]:
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)


Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
